# Project Title
Real-time Anomaly Detection in Financial Transactions


## Authors and Team

- **Author 1**: Ferris Atassi, Developer
- **Author 2**: Charles Hang, Developer

# Executive Summary

- [Decisions to be impacted]
- [Business value]
- [Data assets]

- [Literature Review]

Machine learning and artificial intelligence methods have been used to detect credit card fraud for at least 30 years. Bolton and Hand’s 2001 paper “Unsupervised Profiling Methods for Fraud Detection” outlines how supervised methods were used in the 1990s to detect fraudulent transactions. Reflecting the overall popularity of neural networks in that decade, neural networks were used in supervised learning models for fraudulent transaction detection. Building upon that work, Bolton and Hand outlined how unsupervised methods such as clustering and nearest neighbor methods could be used to build unsupervised learning models to detect fraudulent transactions. Their proposed models, “Peer Group Analysis” and “Break Point Analysis,” grouped accounts into peer groups and analyzed changes in their behavior over time, looking for moments (“break points”) in which individual accounts’ behavior suddenly diverged from that of their peers, which could be indicative of fraud.

Bhattacharyya et al.’s 2011 paper “Data mining for credit card fraud: A comparative study” outlined additional fraud detection methods being used in industry around that time. The paper compared the performance of three methods–logistic regression, random forests, and support vector machines. All three methods had strong performances, with precision scores around 0.8. However, these high precision scores were generated through extremely high specificity values (all around 0.98), meaning that all three methods were successful in labeling 98% of legitimate transactions as legitimate. However, all three methods had much lower sensitivity values (around 0.5 to 0.7), indicating that they had much more difficulty in detecting fraudulent transactions from the overall pool of transactions. 

(Insert table image here)

A preference for higher specificity and lower sensitivity might have business value, since it prevents customers from regularly having their transactions declined incorrectly, thus reducing customer friction and improving customer satisfaction. However, it does also shift the burden of fraud onto credit card issuers, since it means allowing more fraudulent transactions to go through, which later have to be reversed, a cost usually incurred by the issue. In that context, it was notable that random forest models managed the highest sensitivity score of the three models (0.727) while also maintaining the highest specificity (0.987). Because of this, one of the models used in our project was the isolation random forest model, a variant of the random forest model outlined in Bhattacharyya et al.’s paper, which we hoped would have even stronger performance.



# Data Preprocessing

- Data Description -- summary of datasets and visualizations of input data
- Introduce the data cleaning metrics you used in the process and outlier detection.
Data consists of two types of datasets, with both test and training verions of each, resulting in total: 4 data sets: train_identity, test_identity, train_transaction, and test_transaction. They key feature we are looking to analyze is the 'isFraud' field, indicating whether a transaction has been flagged as fraudulent or not. The first step was to combine the test and train datasets into their own, resulting in a train and test dataset. 

The Train and Test datasets consists of multiple groupings of features: TransactionID: ID of the transaction, TransactionDT: timestamp of transaction, TransactionAMT: transaction amount, ProductCD: product code, card1-6: card type and other information, addr1-2: billing address of transaction, dist1-2: distance from billing adress to address of purchase, P_emaildomain: purchaser email domain, R_emaildomain: reciever email domain, C1-14: feature that is counted across different transactions (meaning not given in dataset), D1-15: times between transactions (meaning not given in dataset), M1-9: Matches of fields for each transaction (meaning not given in dataset), V1-???: (meaning not derived yet), id_01-id_38: identification information accumulated by anti-fraud software that developed dataset, DeviceType: type of device transaction was done on, and DeviceInfo: information of device. isFraud is a field only present in the train dataset flagging whether a transaction is considered to be fraud or not (0-not fraud, 1-fraud). 

Current preliminary visualizations include: counts of fraudulent vs non-fraudulent transactions (bar chart), transaction amounts of fraudulent transactions (histogram), Zscores of fraudulent transaction amounts (histogram), --ADD MORE--



# Model Updates

- Include what models you are using or planning to use to support your decision.
- A writeup of machine learning workflow as Machine Learning Morphism (MLM).

# Source Code

In [ ]:
https://github.com/ferrisatassi1/IEEE_Fraud_Detection_Atassi_Hang

# Next Steps

Your plan of how to continue or improve your methods/models and the timeline of next steps.

# Sources

https://scholar.google.com/scholar?q=R.+J.+Bolton+and+D.+J.+Hand.+Unsupervised+profiling+methods+for+fraud+detection.&hl=en&as_sdt=0&as_vis=1&oi=scholart

https://www.dataversity.net/a-brief-history-of-machine-learning/

https://www.sciencedirect.com/science/article/pii/S0167923610001326